In [1]:
import os
import glob
import random
import tempfile

import cv2
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import pandas as pd
import seaborn as sn
import tensorflow as tf
import wandb
from ipywidgets import IntSlider, interact
from matplotlib import animation, rc
from matplotlib.patches import PathPatch, Rectangle
from matplotlib.path import Path
from scipy import ndimage
from scipy.ndimage import zoom
from sklearn.metrics import (classification_report, confusion_matrix,
                             accuracy_score, precision_score, recall_score,
                             f1_score, roc_auc_score, roc_curve)
from sklearn.model_selection import (StratifiedShuffleSplit, StratifiedKFold)
from sklearn.utils import class_weight
from tensorflow.keras import Input, Model
from tensorflow.keras.callbacks import (EarlyStopping, ModelCheckpoint,
                                        ReduceLROnPlateau)
from tensorflow.keras.layers import (BatchNormalization, Conv3D, Dense,
                                     Dropout, GlobalAveragePooling3D,
                                     MaxPool3D, Rescaling)
from tensorflow.keras.optimizers import Adam
from wandb.keras import WandbCallback
import torch
import gc
import torch.nn as nn
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
import seaborn as sns

In [13]:
seed = 27

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [ ]:
wandb.config = {
    'learning_rate': 1e-3,
    'min_learning_rate': 1e-5,
    'batch_size': 8,
    'test_batch_size': 1,
    'img_size': 110,
    'depth': 110,
    'n_classes': 2
}

config = wandb.config

In [1]:
df = pd.read_csv('./dataset/ADNI_MERGE_raw.csv')
ad_paths = df.loc[df['DX'] == 'Dementia', 'IMAGEPATH'].tolist()[:200]
cn_paths = df.loc[df['DX'] == 'CN', 'IMAGEPATH'].tolist()[:200]
mci_paths = df.loc[df['DX'] == 'MCI', 'IMAGEPATH'].tolist()[:200]
ad_labels = np.array([2 for _ in range(len(ad_paths))])
cn_labels = np.array([0 for _ in range(len(cn_paths))])
mci_labels = np.array([1 for _ in range(len(mci_paths))])
labels = np.concatenate((np.concatenate((ad_labels, cn_labels), axis=0), mci_labels), axis = 0)
paths = np.concatenate((np.concatenate((ad_paths, cn_paths), axis=0), mci_paths), axis=0)

Number of CN images : 605
Number of AD images : 365


In [6]:
CLASS_NAMES = ['AD', 'CN']

In [2]:
train_indices, test_indices = next(
    StratifiedShuffleSplit(1, train_size=0.8, random_state=seed).split(
        paths, labels
    )
)

tmp_labels = [labels[idx] for idx in train_indices]
tmp_paths = [paths[idx] for idx in train_indices]

tmp_train_indices, tmp_val_indices = next(
    StratifiedShuffleSplit(5, train_size=0.8, random_state=seed).split(
        tmp_paths, tmp_labels
    )
)

tmp_train_paths = [tmp_paths[idx] for idx in tmp_train_indices]
tmp_val_paths = [tmp_paths[idx] for idx in tmp_val_indices]

train_indices = [paths.tolist().index(path) for path in tmp_train_paths]
val_indices = [paths.tolist().index(path) for path in tmp_val_paths]

# Check if there are no common indices
assert bool(set(train_indices) & set(val_indices)) is False
assert bool(set(train_indices) & set(test_indices)) is False
assert bool(set(val_indices) & set(test_indices)) is False

print(
    'Number of samples:\n'
    f'train: {len(train_indices)} ({round(float(len(train_indices)) / float(len(labels)) * 100.0)}% of the dataset)\n'
    f'validation: {len(val_indices)} ({round(float(len(val_indices)) / float(len(labels)) * 100.0)}% of the dataset)\n'
    f'test: {len(test_indices)} ({round(float(len(test_indices)) / float(len(labels)) * 100.0)}% of the dataset)'
)   



Number of samples:
train: 653 (64% of the dataset)
validation: 163 (16% of the dataset)
test: 204 (20% of the dataset)



In [9]:
class DataGenerator(tf.keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, indices, paths, labels, batch_size=4, dim=(128, 128, 64),
                 n_classes=2, shuffle=True, transform=None):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.paths = paths
        self.labels = labels
        self.indices = indices
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.transform = transform
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.indices) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # Find list of IDs
        indices_temp = [self.indices[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(indices_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.indices))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indices_temp):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, 1))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(indices_temp):
            # Store sample
            volume = np.load(self.paths[ID])['data']
            volume = (volume.astype('float32') / 255.0).astype('float32')
            if self.transform is not None:
                volume = self.transform(volume)
            X[i,] = np.expand_dims(volume, axis=3)

            # Store class
            y[i] = self.labels[ID]

        return X, tf.keras.utils.to_categorical(y, num_classes=self.n_classes)

In [10]:
training_generator = DataGenerator(
    train_indices,
    paths,
    labels,
    batch_size=config['batch_size'],
    dim=(config['img_size'], config['img_size'], config['depth']),
    shuffle=True,
)

validation_generator = DataGenerator(
    val_indices, 
    paths,
    labels, 
    batch_size=config['batch_size'],
    dim=(config['img_size'], config['img_size'], config['depth']),
    shuffle=True,
)

test_generator = DataGenerator(
    test_indices,
    paths,
    labels, 
    batch_size=config['test_batch_size'],
    dim=(config['img_size'], config['img_size'], config['depth']),
    shuffle=False,
)

In [3]:

import torch
import torch.nn as nn

class MRI_Network(nn.Module):
    def __init__(self, input_channels, input_shape, output_size, lstm_layers=1):
        super(MRI_Network, self).__init__()

        self.cnn_model = _3DCNNModel()

        with torch.no_grad():
            dummy_input = torch.zeros((1, input_channels, *input_shape))
            cnn_output_shape = self.cnn_model(dummy_input).shape[1:]
            lstm_input_dimensions = torch.prod(torch.tensor(cnn_output_shape)).item()

        self.lstm = nn.LSTM(lstm_input_dimensions, lstm_input_dimensions, lstm_layers)

        self.num_layers = lstm_layers
        self.hidden_dimensions = lstm_input_dimensions

    def forward(self, x):
        x = self.cnn_model(x)

        x = x.view(x.size(0), -1)  
        x = x.unsqueeze(0) 

        # Pass through LSTM
        x, _ = self.lstm(x)  

        x = x[-1]

        return x



class _3DCNNModel(nn.Module):
    def __init__(self):
        super().__init__()

        # First convolutional block
        self.conv_block1 = nn.Sequential(
            nn.Conv3d(1, 5, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(5, 5, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=2),
            nn.Dropout(0.3)
        )

        # Second convolutional block
        self.conv_block2 = nn.Sequential(
            nn.Conv3d(5, 16, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(16, 16, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=2),
            nn.Dropout(0.3)
        )

        # Third convolutional block
        self.conv_block3 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(32, 32, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(32, 32, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=2),
            nn.Dropout(0.4)
        )

        # Fourth convolutional block
        self.conv_block4 = nn.Sequential(
            nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 64, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.Conv3d(64, 64, kernel_size=(3, 3, 3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=(2, 2, 2), stride=2),
            nn.Dropout(0.4)
        )

        self.conv_block4.name = 'conv_4'
        # Flatten layer
        self.flatten = nn.Flatten()

        self.fc1 = nn.Linear(64 * 6 * 6 * 6, 256)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(256, 128)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        # Pass the input through the convolutional blocks
        x = self.conv_block1(x)
        x = self.conv_block2(x)
        x = self.conv_block3(x)
        x = self.conv_block4(x)

        # Flatten the output of the convolutional layers
        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)

        return x


In [ ]:
from torchinfo import summary
import torch


input_channels = 1
input_shape = (110, 110, 110)  
output_size = 2  
net = MRI_Network(input_channels, input_shape, output_size)
print(net)


MRI_model = MRI_Network(input_channels, input_shape, output_size)

summary(MRI_model, input_size=(1, 1, 110, 110, 110))  


In [18]:
MRI_model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=config['learning_rate']),
    metrics=['acc'],
)

MRI_model.fit(
    training_generator,
    validation_data=validation_generator,
    epochs=config['epochs'],
    verbose=2,
    callbacks=[
        ReduceLROnPlateau(
            monitor='val_loss',
            mode='min',
            patience=7,
            factor=0.5,
            min_lr=config['min_learning_rate'],
        ),
        EarlyStopping(monitor='val_loss', mode='min',
                      patience=config['epochs'], restore_best_weights=True),
        WandbCallback(mode='min'),
    ],
    workers=6
)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
66/66 - 227s - loss: 0.1723 - acc: 0.9489 - val_loss: 1.7022 - val_acc: 0.6016
Epoch 2/100
66/66 - 211s - loss: 0.1838 - acc: 0.9432 - val_loss: 16.1324 - val_acc: 0.3203
Epoch 3/100
66/66 - 211s - loss: 0.2915 - acc: 0.9072 - val_loss: 0.9951 - val_acc: 0.8047
Epoch 4/100
66/66 - 212s - loss: 0.1571 - acc: 0.9489 - val_loss: 1.8725 - val_acc: 0.7422
Epoch 5/100
66/66 - 213s - loss: 0.1658 - acc: 0.9451 - val_loss: 0.9304 - val_acc: 0.7891
Epoch 6/100
66/66 - 215s - loss: 0.1429 - acc: 0.9621 - val_loss: 8.4826 - val_acc: 0.3281
Epoch 7/100
66/66 - 220s - loss: 0.1598 - acc: 0.9470 - val_loss: 1.3930 - val_acc: 0.7969
Epoch 8/100
66/66 - 214s - loss: 0.1290 - acc: 0.9697 - val_loss: 8.8552 - val_acc: 0.3203
Epoch 9/100
66/66 - 212s - loss: 0.1583 - acc: 0.9640 - val_loss: 0.5820 - val_acc: 0.8516
Epoch 10/100
66/66 - 212s - loss: 0.1808 - acc: 0.9318 - val_loss: 2.8511 - val_acc: 0.5469
Epoch 11/100
66/66 - 212s - loss: 0.2087 - acc: 0.9280 - val_loss: 10.4151 - val_acc: 0.

wandb: Network error (ReadTimeout), entering retry loop.


66/66 - 210s - loss: 0.0401 - acc: 0.9962 - val_loss: 0.3347 - val_acc: 0.9141
Epoch 74/100
66/66 - 208s - loss: 0.0448 - acc: 0.9943 - val_loss: 0.3965 - val_acc: 0.9141
Epoch 75/100
66/66 - 211s - loss: 0.0502 - acc: 0.9924 - val_loss: 0.4301 - val_acc: 0.9062
Epoch 76/100
66/66 - 207s - loss: 0.0572 - acc: 0.9886 - val_loss: 0.4040 - val_acc: 0.9062
Epoch 77/100
66/66 - 210s - loss: 0.0804 - acc: 0.9830 - val_loss: 0.3709 - val_acc: 0.9062
Epoch 78/100
66/66 - 208s - loss: 0.0734 - acc: 0.9811 - val_loss: 0.4368 - val_acc: 0.9062
Epoch 79/100
66/66 - 207s - loss: 0.0991 - acc: 0.9848 - val_loss: 0.3505 - val_acc: 0.9141
Epoch 80/100
66/66 - 207s - loss: 0.0397 - acc: 0.9924 - val_loss: 0.3654 - val_acc: 0.9141
Epoch 81/100
66/66 - 208s - loss: 0.0356 - acc: 0.9981 - val_loss: 0.4823 - val_acc: 0.8984
Epoch 82/100
66/66 - 212s - loss: 0.0441 - acc: 0.9943 - val_loss: 0.4793 - val_acc: 0.8984
Epoch 83/100
66/66 - 209s - loss: 0.0719 - acc: 0.9886 - val_loss: 0.3762 - val_acc: 0.9062
E

In [ ]:
device = 'cuda'

In [7]:
MRI_model.eval()
with torch.no_grad():
  MRI_features = MRI_model.relu2(MRI_model.fc2(MRI_model.relu1(MRI_model.fc1(MRI_model.flatten(MRI_model.conv_block4(MRI_model.conv_block3(MRI_model.conv_block2(MRI_model.conv_block1(((training_generator.to(device))))))))))))

In [ ]:
in_size = 204*5

### MRI only

In [1]:
MRI_model = MRI_Network()
MRI_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(MRI_model.parameters(), lr=0.001)


num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(MRI_features, training_generator)):
    print(f"Fold {fold + 1}/{num_folds}")

    train_features, val_features = MRI_features[train_index], MRI_features[val_index]
    train_labels, val_labels = training_generator[train_index], training_generator[val_index]

    num_epochs = 50
    for epoch in range(num_epochs):
      MRI_model.train()
      optimizer.zero_grad()
      outputs = MRI_model(train_features.to(device))
      loss = criterion(outputs, torch.argmax(train_labels.to(device), dim=1))
      loss.backward(retain_graph=True)
      optimizer.step()

      print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

    MRI_model.eval()
    with torch.no_grad():
        val_outputs = MRI_model(val_features)
        _, predicted = torch.max(val_outputs, 1)
        correct = (predicted.to(device) == torch.argmax(val_labels.to(device), dim=1)).sum().item()
        total = val_labels.size(0)
        accuracy = correct / total * 100
        print(f"Accuracy on fold {fold + 1}: {accuracy}%")
        



       
Fold 1/5
Epoch 1/50, Loss: 0.7362676858901978
Epoch 2/50, Loss: 0.7169378995895386
Epoch 3/50, Loss: 0.7166941165924072
Epoch 4/50, Loss: 0.6989967226982117
Epoch 5/50, Loss: 0.6887317299842834
Epoch 6/50, Loss: 0.6893790364265442
Epoch 7/50, Loss: 0.6894196271896362
Epoch 8/50, Loss: 0.6837486028671265
Epoch 9/50, Loss: 0.6770045757293701
Epoch 10/50, Loss: 0.6756261587142944
Epoch 11/50, Loss: 0.6746048927307129
Epoch 12/50, Loss: 0.6698505878448486
Epoch 13/50, Loss: 0.6652263402938843
Epoch 14/50, Loss: 0.6632023453712463
Epoch 15/50, Loss: 0.6606860160827637
Epoch 16/50, Loss: 0.6562567353248596
Epoch 17/50, Loss: 0.6534498333930969
Epoch 18/50, Loss: 0.6505505442619324
Epoch 19/50, Loss: 0.6462287902832031
Epoch 20/50, Loss: 0.6422210335731506
Epoch 21/50, Loss: 0.63814377784729
Epoch 22/50, Loss: 0.6330428719520569
Epoch 23/50, Loss: 0.6281920075416565
Epoch 24/50, Loss: 0.6231706738471985
Epoch 25/50, Loss: 0.6177356243133545
Epoch 26/50, Loss: 0.6131470203399658
Epoch

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

def print_test_metrics(true_labels, predicted_labels):
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    
    tn, fp, fn, tp = confusion_matrix(true_labels, predicted_labels).ravel()
    
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    
    print("Accuracy\t", accuracy)
    print("Precision\t", precision)
    print("Recall\t", recall)
    print("F1 Score\t", f1)
    print("Sensitivity\t", sensitivity)
    print("Specificity\t", specificity)

In [22]:
y_pred = [np.argmax(x) for x in MRI_model.predict(test_generator, batch_size=config['batch_size'])]
y_test = [labels[idx] for idx in test_indices]

In [4]:
print("Test Metrics for MRI only:\n")
print_test_metrics(y_test, y_pred)

Test Metrics for MRI only:

Accuracy	83.34
Precision	85.11
Recall		83.86
F1 Score	84.48
Sensitivity	69.42
Specificity	88.66


### MRI + Demo

In [ ]:
democog_data = pd.read_csv('./processed_adni_merge_normalized.csv')

In [ ]:
demographic_features = ['AGE', 'PTGENDER', 'PTEDUCAT']

X_demographic = democog_data[demographic_features].values
scaler_demo = StandardScaler()
X_demo = scaler_demo.fit_transform(X_demographic)
X_demo_tensor = torch.FloatTensor(X_demo)

In [ ]:
X_demo_train = X_demo_tensor[:int(in_size*0.8)]
X_demo_test = X_demo_tensor[int(in_size*0.8):]

In [ ]:
class DemographicModel(nn.Module):
    def __init__(self):
        super(DemographicModel, self).__init__()
        self.fc1 = nn.Linear(17, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return x

In [ ]:
demo_model = DemographicModel()
summary(demo_model, input_size=(1, 17))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 32]             576
              ReLU-2                [-1, 1, 32]               0
            Linear-3                [-1, 1, 16]             528
              ReLU-4                [-1, 1, 16]               0
Total params: 1,104
Trainable params: 1,104
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [ ]:
demo_model.eval()
demo_features = demo_model.relu2(demo_model.fc2(demo_model.relu1(demo_model.fc1(X_demo_train.to(device)))))
print(demo_features)

tensor([[0.0000, 0.0000, 0.4356,  ..., 0.1656, 0.2368, 0.0881],
        [0.0000, 0.0000, 0.6251,  ..., 0.2428, 0.2526, 0.0098],
        [0.0000, 0.0232, 0.4791,  ..., 0.1128, 0.2478, 0.2578],
        ...,
        [0.0672, 0.0000, 0.2384,  ..., 0.0113, 0.0000, 0.0000],
        [0.4158, 0.0000, 0.2405,  ..., 0.0000, 0.0000, 0.0000],
        [0.3263, 0.0891, 0.3835,  ..., 0.1560, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)


In [ ]:
concatenated_features = torch.cat((MRI_features, demo_features), dim=1)

In [ ]:
class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.fc1 = nn.Linear(128 + 16, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [7]:
mri_demo_model = FusionModel()
mri_demo_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mri_demo_model.parameters(), lr=0.001)


num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(concatenated_features, training_generator)):
    print(f"Fold {fold + 1}/{num_folds}")

    train_features, val_features = concatenated_features[train_index], concatenated_features[val_index]
    train_labels, val_labels = training_generator[train_index], training_generator[val_index]

    num_epochs = 50
    for epoch in range(num_epochs):
      mri_demo_model.train()
      optimizer.zero_grad()
      outputs = mri_demo_model(train_features.to(device))
      loss = criterion(outputs, torch.argmax(train_labels.to(device), dim=1))
      loss.backward(retain_graph=True)
      optimizer.step()

      print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

    mri_demo_model.eval()
    with torch.no_grad():
        val_outputs = mri_demo_model(val_features)
        _, predicted = torch.max(val_outputs, 1)
        correct = (predicted.to(device) == torch.argmax(val_labels.to(device), dim=1)).sum().item()
        total = val_labels.size(0)
        accuracy = correct / total * 100
        print(f"Accuracy on fold {fold + 1}: {accuracy}%")


Fold 1/5
Epoch 1/50, Loss: 0.7362676858901978
Epoch 2/50, Loss: 0.7169378995895386
Epoch 3/50, Loss: 0.7166941165924072
Epoch 4/50, Loss: 0.6989967226982117
Epoch 5/50, Loss: 0.6887317299842834
Epoch 6/50, Loss: 0.6893790364265442
Epoch 7/50, Loss: 0.6894196271896362
Epoch 8/50, Loss: 0.6837486028671265
Epoch 9/50, Loss: 0.6770045757293701
Epoch 10/50, Loss: 0.6756261587142944
Epoch 11/50, Loss: 0.6746048927307129
Epoch 12/50, Loss: 0.6698505878448486
Epoch 13/50, Loss: 0.6652263402938843
Epoch 14/50, Loss: 0.6632023453712463
Epoch 15/50, Loss: 0.6606860160827637
Epoch 16/50, Loss: 0.6562567353248596
Epoch 17/50, Loss: 0.6534498333930969
Epoch 18/50, Loss: 0.6505505442619324
Epoch 19/50, Loss: 0.6462287902832031
Epoch 20/50, Loss: 0.6422210335731506
Epoch 21/50, Loss: 0.63814377784729
Epoch 22/50, Loss: 0.6330428719520569
Epoch 23/50, Loss: 0.6281920075416565
Epoch 24/50, Loss: 0.6231706738471985
Epoch 25/50, Loss: 0.6177356243133545
Epoch 26/50, Loss: 0.6131470203399658
Epoch 27/50, 

In [ ]:
y_pred = [np.argmax(x) for x in mri_demo_model.predict(test_generator, batch_size=config['batch_size'])]
y_test = [labels[idx] for idx in test_indices]

In [8]:
print("Test Metrics for MRI + Demographic data:\n")
print_test_metrics(y_test, y_pred)

Test Metrics for MRI + Demographic data:

Accuracy	85.66
Precision	85.29
Recall		84.75
F1 Score	85.02
Sensitivity	70.33
Specificity	88.74


### MRI + Cognitive 

In [ ]:
cognitive_features = ['ADAS13', 'TAU', 'PTAU', 'CDRSB', 'MMSE', 'RAVLT_immediate', 'RAVLT_learning',
                         'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'FAQ', 'MOCA', 'Hippocampus',
                         'APOE4', 'FDG']


X_cognitive = democog_data[demographic_features].values
scaler_cog = StandardScaler()
X_cog = scaler_cog.fit_transform(X_cognitive)
X_cog_tensor = torch.FloatTensor(X_cog)

In [ ]:
X_cog_train = X_cog_tensor[:int(in_size*0.8)]
X_cog_test = X_cog_tensor[int(in_size*0.8):]

In [ ]:
class CognitiveModel(nn.Module):
    def __init__(self):
        super(CognitiveModel, self).__init__()
        self.fc1 = nn.Linear(17, 32)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        return x

In [ ]:
cog_model = CognitiveModel()
summary(cog_model, input_size=(1, 17))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 32]             576
              ReLU-2                [-1, 1, 32]               0
            Linear-3                [-1, 1, 16]             528
              ReLU-4                [-1, 1, 16]               0
Total params: 1,104
Trainable params: 1,104
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.01
----------------------------------------------------------------


In [ ]:
cog_model.eval()
cog_features = cog_model.relu2(cog_model.fc2(cog_model.relu1(cog_model.fc1(X_cog_train.to(device)))))
print(cog_features)

tensor([[0.0000, 0.0000, 0.4356,  ..., 0.1656, 0.2368, 0.0881],
        [0.0000, 0.0000, 0.6251,  ..., 0.2428, 0.2526, 0.0098],
        [0.0000, 0.0232, 0.4791,  ..., 0.1128, 0.2478, 0.2578],
        ...,
        [0.0672, 0.0000, 0.2384,  ..., 0.0113, 0.0000, 0.0000],
        [0.4158, 0.0000, 0.2405,  ..., 0.0000, 0.0000, 0.0000],
        [0.3263, 0.0891, 0.3835,  ..., 0.1560, 0.0000, 0.0000]],
       device='cuda:0', grad_fn=<ReluBackward0>)


In [ ]:
concatenated_features = torch.cat((MRI_features, cog_features), dim=1)

In [ ]:
class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.fc1 = nn.Linear(128 + 16, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        return x

In [11]:
mri_cog_model = FusionModel()
mri_cog_model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mri_cog_model.parameters(), lr=0.001)


num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

for fold, (train_index, val_index) in enumerate(skf.split(concatenated_features, training_generator)):
    print(f"Fold {fold + 1}/{num_folds}")

    train_features, val_features = concatenated_features[train_index], concatenated_features[val_index]
    train_labels, val_labels = training_generator[train_index], training_generator[val_index]

    num_epochs = 50
    for epoch in range(num_epochs):
      mri_cog_model.train()
      optimizer.zero_grad()
      outputs = mri_cog_model(train_features.to(device))
      loss = criterion(outputs, torch.argmax(train_labels.to(device), dim=1))
      loss.backward(retain_graph=True)
      optimizer.step()

      print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {loss.item()}")

    mri_cog_model.eval()
    with torch.no_grad():
        val_outputs = mri_cog_model(val_features)
        _, predicted = torch.max(val_outputs, 1)
        correct = (predicted.to(device) == torch.argmax(val_labels.to(device), dim=1)).sum().item()
        total = val_labels.size(0)
        accuracy = correct / total * 100
        print(f"Accuracy on fold {fold + 1}: {accuracy}%")



Fold 1/5
Epoch 1/50, Loss: 0.7362676858901978
Epoch 2/50, Loss: 0.7169378995895386
Epoch 3/50, Loss: 0.7166941165924072
Epoch 4/50, Loss: 0.6989967226982117
Epoch 5/50, Loss: 0.6887317299842834
Epoch 6/50, Loss: 0.6893790364265442
Epoch 7/50, Loss: 0.6894196271896362
Epoch 8/50, Loss: 0.6837486028671265
Epoch 9/50, Loss: 0.6770045757293701
Epoch 10/50, Loss: 0.6756261587142944
Epoch 11/50, Loss: 0.6746048927307129
Epoch 12/50, Loss: 0.6698505878448486
Epoch 13/50, Loss: 0.6652263402938843
Epoch 14/50, Loss: 0.6632023453712463
Epoch 15/50, Loss: 0.6606860160827637
Epoch 16/50, Loss: 0.6562567353248596
Epoch 17/50, Loss: 0.6534498333930969
Epoch 18/50, Loss: 0.6505505442619324
Epoch 19/50, Loss: 0.6462287902832031
Epoch 20/50, Loss: 0.6422210335731506
Epoch 21/50, Loss: 0.63814377784729
Epoch 22/50, Loss: 0.6330428719520569
Epoch 23/50, Loss: 0.6281920075416565
Epoch 24/50, Loss: 0.6231706738471985
Epoch 25/50, Loss: 0.6177356243133545
Epoch 26/50, Loss: 0.6131470203399658
Epoch 27/50, 

In [ ]:
y_pred = [np.argmax(x) for x in mri_cog_model.predict(test_generator, batch_size=config['batch_size'])]
y_test = [labels[idx] for idx in test_indices]

In [12]:
print("Test Metrics for MRI + Cognitive data:\n")
print_test_metrics(y_test, y_pred)

Test Metrics for MRI + Cognitive data:

Accuracy	89.12
Precision	87.77
Recall		86.87
F1 Score	87.32
Sensitivity	71.76
Specificity	89.82
